In [1]:
%matplotlib notebook

# %qtconsole

import logging
import time
from importlib import reload

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

import qcodes as qc
from qcodes.dataset.experiment_container import new_experiment

In [2]:
from pytopo.qctools import instruments as instools
from pytopo.qctools.instruments import create_inst, add2station

from pytopo.sweep.base import Nest, Chain
from pytopo.sweep.decorators import getter, setter
from pytopo.sweep import sweep, do_experiment, hardsweep, measure

In [3]:
pwd

'd:\\OneDrive\\Setups\\LK1\\LK1_code\\Code\\LK1\\LK1\\Experiments'

# Define measurement functions

In [4]:
def setup_frq_sweep(fstart, fstop, fpts, chan='S21', bw=None, navgs=None, pwr=None):
    """
    Setup a VNA trace.
    
    assumes that a channel with name chan is already created.
    """
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, chan)
    
    fvals = np.linspace(fstart, fstop, fpts)
    trace.start(fstart)
    trace.stop(fstop)
    trace.npts(fpts)
    if navgs is not None:
        trace.avg(navgs)
    if bw is not None:
        trace.bandwidth(bw)
    if pwr is not None:
        trace.power(pwr)
    trace.autoscale()
    
    if not vna.rf_power():
        vna.rf_on()
    
    return fvals

def take_trace(chan='S21', plot=False):
    """
    Get the data of a currently measured trace.
    The trace has to be setup already.
    
    Returns magnitude (in dB) and phase (in rad).
    
    If plot is true, make a simple plot of the magnitude vs frequency.
    """
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, chan)
    
    fvals = np.linspace(trace.start(), trace.stop(), trace.npts())
    mag, phase = trace.trace_mag_phase()
    
    if plot:
        fig, ax = plt.subplots(1, 1)
        ax.plot(fvals*1e-9, 20*np.log10(mag))
        ax.grid(dashes=[1,1])
        ax.set_xlabel('Frequency (GHz)')
        ax.set_ylabel('Magnitude (dBm)')
        
    return mag, phase


@hardsweep(
    ind=[('frequency', 'Hz', 'array')], 
    dep=[('amplitude', '', 'array'), ('phase', 'rad', 'array')]
)
def vna_frequency_sweep(*arg, **kw):
    """
    Measurement function for a simple VNA trace for pytopo.sweep.
    
    All arguments will be passed to setup_trace.
    Return data is as returned by take_trace.
    """
    fvals = setup_frq_sweep(*arg, **kw)
    mag, phase = take_trace(plot=False)
    
    return fvals, np.vstack((mag, phase))


@hardsweep(
    ind=[('frequency', 'Hz', 'array')], 
    dep=[('magnitude', 'dB', 'array'), ('phase', 'rad', 'array')]
)
def vna_frequency_sweep_dB(*arg, **kw):
    """
    Measurement function for a simple VNA trace for pytopo.sweep.
    
    All arguments will be passed to setup_trace.
    Return data is as returned by take_trace.
    """
    fvals = setup_frq_sweep(*arg, **kw)
    mag, phase = take_trace(plot=False)
    
    return fvals, np.vstack((20*np.log10(mag), phase))


def setup_zerospan_trace(frq, npts, chan='S21', span=1.):
    """
    Setup the VNA to measure a trace around <frq> with <npts> samples
    in a very narray frequency span.
    """
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, chan)
    
    trace.center(frq)
    trace.span(span)
    trace.npts(npts)
    
    trace.autoscale()
    
    if not vna.rf_power():
        vna.rf_on()


@hardsweep(
    ind=[('iteration', '', 'array')], 
    dep=[('amplitude', '', 'array'), ('phase', 'rad', 'array')]
)
def vna_timetrace(*arg, **kw):
    """
    Measure a 'time trace', emulated by a narrow span trace with many points.
    """
    setup_zerospan_trace(*arg, **kw)
    mag, phase = take_trace(plot=False)
    
    return np.arange(mag.size), np.vstack((mag, phase))


@hardsweep(
    ind=[('iteration', '', 'array')], 
    dep=[('magnitude', 'dB', 'array'), ('phase', 'rad', 'array')]
)
def vna_timetrace(*arg, **kw):
    """
    Measure a 'time trace', emulated by a narrow span trace with many points.
    """
    setup_zerospan_trace(*arg, **kw)
    mag, phase = take_trace(plot=False)
    
    return np.arange(mag.size), np.vstack((20*np.log10(mag), phase))

# Init station

## Global variables

In [5]:
SAMPLE = "W05_Combo_11_RF04_CD20190503"

## Init instruments

In [6]:
%run -i "D:\OneDrive\Setups\LK1\LK1_code\Code\LK1\LK1\init_station_VNA.py"

Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062102038, firmware:2.92) in 0.52s
Initialized IVVI-rack in 0.03s
Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:124050266, firmware:2.6.04.000) in 0.12s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110430, firmware:3.1.19.15-3.50.124.67) in 0.04s


In [7]:
station.ivvi.set_dacs_zero()

In [8]:
station.components

{'vna': <ZNB: vna>,
 'ivvi': <IVVI: ivvi>,
 'TWPA': <RohdeSchwarz_SGS100A: TWPA>,
 'mgnt': <MercuryiPS: mgnt>}

In [9]:
numdacs = int(station.ivvi._numdacs)
for i in range(1, numdacs+1):
    station.ivvi.parameters['dac{}'.format(i)].step = 10
    station.ivvi.parameters['dac{}'.format(i)].post_delay = 0.001

In [11]:
station.vna.add_channel('S21')
station.vna.rf_off()

## Setup-specific DAC assignments

In [16]:
dac_cutterD = station.ivvi.dac2
dac_cutterC = station.ivvi.dac5
dac_cutterB = station.ivvi.dac4
dac_cutterA = station.ivvi.dac11

#if we are measuring device A
dac_bottomA = station.ivvi.dac13
#dac_wrapLA = station.ivvi.dac13
#dac_wrapRA = station.ivvi.dac3

#if we are measuring deviceB
dac_bottomB = station.ivvi.dac6
#dac_wrapLB = station.ivvi.dac4
#dac_wrapRB = station.ivvi.dac6

#if we are measuring deviceC
dac_bottomC = station.ivvi.dac1
#dac_wrapLC = station.ivvi.dac1
#dac_wrapRC = station.ivvi.dac11

#if we are measuring deviceD
dac_bottomD = station.ivvi.dac3
#dac_wrapLD = station.ivvi.dac5
#dac_wrapRD = station.ivvi.dac2

@setter(('plungersB', 'mV'),)
def set_plungersB(val):
    """
    Set both plungers (left and right) to val (in mV).
    """
    dac_wrapLB(val)
    dac_wrapRB(val)
    
    return val

@setter(('plungersD', 'mV'),)
def set_plungersD(val):
    """
    Set both plungers (left and right) to val (in mV).
    """
    dac_wrapLD(val)
    dac_wrapRD(val)
    
    return val

@setter(('cuttersAll', 'mV'),)
def set_cuttersAll(val):
    """
    Set all four cutters to val (in mV).
    """
    dac_cutterA(val)
    dac_cutterB(val)
    dac_cutterC(val)
    dac_cutterD(val)
    
    return val

@setter(('backgatesAll', 'mV'),)
def set_backgatesAll(val):
    """
    Set all four cutters to val (in mV).
    """
    dac_bottomA(val)
    dac_bottomB(val)
    dac_bottomC(val)
    dac_bottomD(val)
    
    return val


@setter(('plungersAll', 'mV'),)
def set_plungersAll(val):
    """
    Set all plungers (left and right) to val (in mV).
    """
    dac_wrapLD(val)
    dac_wrapRD(val)
    
    dac_wrapLC(val)
    dac_wrapRC(val)
    
    dac_wrapLB(val)
    dac_wrapRB(val)
    
    dac_wrapLA(val)
    dac_wrapRA(val)
    
    return val

In [13]:
time.sleep(1)

In [12]:
@setter(('none', 'mV'),)
def set_none(val):
    """
    Set none (in mV).
    """
    time.sleep(1)
    
    return val

In [ ]:
freqs = np.linspace(3.0e9,7.0e9,4001)
plt.figure()
plt.plot(freqs,mag)
plt.plot(freqs,mag1)

In [ ]:
#check that we can take the trace correctly
setup_frq_sweep(3.0e9, 7.0e9, 4001, bw=1e3, navgs=1, pwr=-60)
mag, phase = take_trace(plot=True)

In [ ]:
setup_frq_sweep(3.0e9, 7.0e9, 4001, bw=1e3, navgs=1, pwr=-60)
mag1, phase1 = take_trace(plot=True)

In [ ]:
twpa_pump.modulation_rf('OFF')

# Testing

## Quick manual trace (without saving)

In [ ]:
setup_frq_sweep(fstart=3e9, fstop=6e9, fpts=1501, pwr=-50)
mag, phase = take_trace(plot=True)

# Take a single VNA trace

In [ ]:
station.components

In [13]:
station.vna.S21.power(-50)
sweep_obj = vna_frequency_sweep_dB(fstart=2.5e9, fstop=8.5e9, fpts=10001, chan='S21')

result = do_experiment(f"VNA/{SAMPLE}", sweep_obj, live_plot=False)

Starting experimental run with id: 164


In [ ]:
station.vna.S21.power(-50)
sweep_obj = vna_frequency_sweep_dB(fstart=3.015e9, fstop=3.025e9, fpts=1001, chan='S21')

result = do_experiment(f"VNA/{SAMPLE}", sweep_obj, live_plot=False)

# Take a gatesweep vs VNA

In [ ]:
station.vna.S21.avg(10)

In [20]:
station.ivvi.set_dacs_zero()

In [ ]:
for freqs in np.arange(2e9,6.1e9,0.5e9):
    station.vna.S21.power(-60)
    sweep_obj = vna_frequency_sweep_dB(fstart=freqs, fstop=freqs+0.5e9, fpts=5001, chan='S21')

    result = do_experiment(f"VNA/{SAMPLE}", sweep_obj, live_plot=False)

In [71]:
station.ivvi.set_dacs_zero()

In [ ]:
station.vna.S21.power(-55)
station.vna.S21.avg(3)
cur_cutter = dac_bottomA()
sweep_obj = sweep(set_backgatesAll, np.arange(-1900.,1900.,10.))(
    vna_frequency_sweep_dB(fstart=3.5e9, fstop=4.75e9, fpts=10001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

Starting experimental run with id: 170


In [54]:
station.vna.S21.power(-55)
station.vna.S21.avg(5)
cur_cutter = dac_cutterA()
sweep_obj = sweep(set_cuttersAll, np.arange(0.,-900.,-10.))(
    vna_frequency_sweep_dB(fstart=3.644e9, fstop=3.646e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

Starting experimental run with id: 157


In [69]:
station.ivvi.dac1(0.)

In [70]:
station.vna.S21.power(-60)
station.vna.S21.avg(2)
cur_cutter = dac_bottomC()
sweep_obj = sweep(station.ivvi.dac1, np.arange(0.,-900.,-2.))(
    vna_frequency_sweep_dB(fstart=4.39e9, fstop=4.41e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

Starting experimental run with id: 163


KeyboardInterrupt: 

In [ ]:
for freqs in np.arange(2.5e9,7.1e9,0.5e9):
    station.vna.S21.power(-60)
    cur_cutter = dac_cutterA()
    sweep_obj = sweep(set_cuttersAll, np.arange(0.,600.0,3))(
        vna_frequency_sweep_dB(fstart=freqs, fstop=freqs+0.5e9, fpts=5001, chan='S21')
    )

    result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [16]:
for i in np.arange(1000):
    station.vna.S21.avg(20)
    station.vna.S21.power(-50)

    sweep_obj = sweep(set_none, np.arange(1000))(
        vna_frequency_sweep_dB(fstart=2.5e9, fstop=7.5e9, fpts=8001, chan='S21')
    )

    result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

Starting experimental run with id: 135
Starting experimental run with id: 136
Starting experimental run with id: 137


KeyboardInterrupt: 

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_bottomA()
sweep_obj = sweep(dac_cutterA, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_cutterB()
sweep_obj = sweep(dac_cutterB, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_bottomB()
sweep_obj = sweep(dac_bottomB, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_cutterC()
sweep_obj = sweep(dac_cutterC, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_bottomC()
sweep_obj = sweep(dac_bottomC, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_cutterD()
sweep_obj = sweep(dac_cutterD, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.S21.power(-60)
cur_cutter = dac_bottomD()
sweep_obj = sweep(dac_bottomD, np.arange(0.,1500.0,50))(
    vna_frequency_sweep_dB(fstart=2.5e9, fstop=5.5e9, fpts=4001, chan='S21')
)

result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.ivvi.set_dacs_zero()

In [ ]:
dac_wrapRD(0)

In [ ]:
vna.S21.power(-60)

#scan from -1V to 1V in plunger while moving cutter
cutters = np.arange(2000.,150.,-2.)
for i in np.arange(0,1849,2):
    dac_cutterD(cutters[i])
    cur_plungers = dac_wrapRD()
    sweep_obj = sweep(set_plungersD, np.arange(cur_plungers,-2000.0,-10))(
        vna_frequency_sweep_dB(fstart=4.74e9, fstop=4.75e9, fpts=601, chan='S21')
    )

    result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)
    cur_plungers = dac_wrapRD()
    dac_cutterD(cutters[i+1])
    sweep_obj = sweep(set_plungersD, np.arange(cur_plungers,0.0,10))(
        vna_frequency_sweep_dB(fstart=4.74e9, fstop=4.75e9, fpts=601, chan='S21')
    )

    result = do_experiment(f"VNA_gatesweep/{SAMPLE}", sweep_obj, live_plot=False)


In [ ]:
station.ivvi.set_dacs_zero()

# Sweep flux vs VNA

In [ ]:
@setter(('magnet_x', 'T'),) 
def set_magnet_x(val):
    """
    Set magnet x
    """
    station.mgnt.x_target(val)
    station.mgnt.ramp(mode='safe')
    return val

@setter(('magnet_y', 'T'),)
def set_magnet_y(val):
    """
    Set magnet y
    """
    station.mgnt.y_target(val)
    station.mgnt.ramp(mode='safe')
    return val

@setter(('magnet_z', 'T'),)
def set_magnet_z(val):
    """
    Set magnet z
    """
    station.mgnt.z_target(val)
    station.mgnt.ramp(mode='safe')
    return val

In [ ]:
station.vna.S21.power(-60)
sweep_obj = sweep(set_magnet_x, np.arange(1.0e-3,4.0e-3,0.01e-3))(
    vna_frequency_sweep_dB(fstart=3.8e9, fstop=4.2e9, fpts=2001, chan='S21')
)

result = do_experiment(f"VNA_Bxsweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
vna.S21.power(-60)
sweep_obj = sweep(set_magnet_z, np.arange(1000.0e-3,0.0e-3-0.1e-3,-10e-3))(
    vna_frequency_sweep_dB(fstart=4.1e9, fstop=4.2e9, fpts=1001, chan='S21')
)

result = do_experiment(f"VNA_Bzsweep/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
station.vna.rf_off()

In [ ]:
station.ivvi.set_dacs_zero()

# Power dependence

In [ ]:
np.linspace(-60., 0., 21)

In [15]:
station.vna.S21.avg(5)

sweep_obj = sweep(station.vna.S21.power, np.linspace(-60., 0., 21))(
   vna_frequency_sweep_dB(fstart=4.0e9, fstop=6.0e9, fpts=10001, chan='S21')
)

result = do_experiment(f"VNA_powersweep/{SAMPLE}", sweep_obj, live_plot=False)



Starting experimental run with id: 166


In [ ]:
@setter(('repetition', '[]'),)
def set_repetition(val):
    """

    Set repetition (to repeat identical sweeps)
    """
    
    time.sleep(0.1)
    return val

In [ ]:
sweep_obj = sweep(vna.S21.power, np.linspace(-60., -20., 10))(sweep(set_repetition, np.linspace(1,100,100))(
    vna_frequency_sweep_dB(fstart=5.3204e9-3e6, fstop=5.3204e9+3e6, fpts=60, chan='S21')
))

result = do_experiment(f"VNA_powersweep_vs_repetition/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
sweep_obj = sweep(vna.S21.power, np.linspace(-60., -20., 10))(sweep(set_repetition, np.linspace(1,200,200))(
    vna_frequency_sweep_dB(fstart=5.347e9, fstop=5.353e9, fpts=60, chan='S21')
))

result = do_experiment(f"VNA_powersweep_vs_repetition/{SAMPLE}", sweep_obj, live_plot=False)

In [ ]:
twpa_pump.on()
vna.S21.avg(1)
vna.S21.power(-55)
setup_zerospan_trace(5.345737e9, 51)
      
sweep_obj = sweep(twpa_pump.power, np.arange(4.0, 10.0, .2))(
     sweep(twpa_pump.frequency, np.arange(7.8e9, 8.1e9, 1.0e6)))(
        measure(get_SNR)
)


result = do_experiment(f"TWPA_SNR/{SAMPLE}", sweep_obj, live_plot=False)
twpa_pump.off()

# TWPA pump tune up

## VNA as function of pump power

In [ ]:
twpa_pump.frequency(8e9)

In [ ]:
station.TWPA.on()
sweep_obj = sweep(station.TWPA.power, np.arange(5, 10, 0.1))(
    vna_frequency_sweep_dB(fstart=3e9, fstop=6e9, fpts=2001, chan='S21', navgs=1)
)

result = do_experiment(f"VNA_vs_pump_power/{SAMPLE}", sweep_obj, live_plot=False)
station.TWPA.off()

## VNA as function of pump frq

In [ ]:
station.TWPA.on()

In [ ]:
station.TWPA.power(8.0)
station.TWPA.on()
sweep_obj = sweep(station.TWPA.frequency, np.arange(7.8e9, 8.2e9, 1e6))(
    vna_frequency_sweep_dB(fstart=3.0e9, fstop=6e9, fpts=2001, chan='S21', navgs=1)
)

result = do_experiment(f"VNA_vs_pump_frequency/{SAMPLE}", sweep_obj, live_plot=False)
station.TWPA.off()

In [ ]:
sweep_obj = sweep(twpa_pump.power, np.arange(9,5,-1))(
    sweep(twpa_pump.frequency, np.arange(7.96e9, 8.08e9, 1e6))(
    vna_frequency_sweep_dB(fstart=4e9, fstop=6e9, fpts=2001, chan='S21', navgs=1)
))

result = do_experiment(f"VNA_vs_pump_frequency/{SAMPLE}", sweep_obj, live_plot=True)
twpa_pump.off()

In [ ]:
twpa_pump.power(5)

sweep_obj = sweep(twpa_pump.frequency, np.arange(7.96e9, 8.08e9, 1e6))(
    vna_frequency_sweep_dB(fstart=4e9, fstop=6e9, fpts=2001, chan='S21', navgs=1)
)

result = do_experiment(f"VNA_vs_pump_frequency/{SAMPLE}", sweep_obj, live_plot=True)

## SNR as function of pump params

In [ ]:
@getter(('signal', 'dB'), ('noise', 'dB'), ('SNR', 'dB'))
def get_SNR():
    time.sleep(0.01)
    mag, phase = take_trace(plot=False)
    sig = np.abs(mag) * np.exp(-1j*phase)
#     I, Q = sig.real, sig.imag
    lin_mean = np.abs(sig.mean())
    lin_std = np.abs(sig.std())
#     lin_mean = (I.mean()**2. + Q.mean()**2.)**.5
#     lin_std = (I.std()**2. + Q.std()**2.)**.5
    return 20*np.log10(lin_mean), 20*np.log10(lin_std), 20*np.log10(lin_mean/lin_std)

In [ ]:
import time

In [ ]:
a,b = 5.95, 7.961
twpa_pump.frequency(b*1e9)
twpa_pump.power(a)
twpa_pump.on()

In [ ]:
station.TWPA.off()

In [ ]:
station.TWPA.frequency(8.0e9)
station.TWPA.power(8.4)
station.TWPA.on()

In [ ]:
twpa_pump.output_rf('ON')
vna.S21.avg(1)
vna.S21.power(-55)
setup_zerospan_trace(3.0185e9, 51)
      
sweep_obj = sweep(twpa_pump.power, np.arange(15.0, 13.5, -.05))(
     sweep(twpa_pump.frequency, np.arange(7.86e9, 7.96e9, 0.5e6)))(
        measure(get_SNR)
)


result = do_experiment(f"TWPA_SNR/{SAMPLE}", sweep_obj, live_plot=False)
twpa_pump.output_rf('OFF')

In [ ]:
station.TWPA.on()
station.vna.S21.avg(1)
station.vna.S21.power(-55)
setup_zerospan_trace(3.0185e9, 51)
      
sweep_obj = sweep(station.TWPA.power, np.arange(3.5, 7.0, .2))(
     sweep(station.TWPA.frequency, np.arange(7.85e9, 8.04e9, 1.0e6)))(
        measure(get_SNR)
)


result = do_experiment(f"TWPA_SNR/{SAMPLE}", sweep_obj, live_plot=False)
twpa_pump.off()

In [ ]:
twpa_pump.on()
vna.S21.avg(1)
vna.S21.power(-55)
setup_zerospan_trace(5.345737e9, 51)
      
sweep_obj = sweep(twpa_pump.power, np.arange(4.0, 10.0, .2))(
     sweep(twpa_pump.frequency, np.arange(7.8e9, 8.1e9, 1.0e6)))(
        measure(get_SNR)
)


result = do_experiment(f"TWPA_SNR/{SAMPLE}", sweep_obj, live_plot=False)
twpa_pump.off()

In [ ]:
twpa_pump.on()
vna.S21.avg(1)
vna.S21.power(-50)
setup_zerospan_trace(5.3136e9, 101)


sweep_obj = sweep(twpa_pump.frequency, np.arange(7.70e9, 7.94e9, 0.5e6))(
     sweep(twpa_pump.power, np.arange(-5.0, 14.0, .1)[::-1]))(
        measure(get_SNR)
)

result = do_experiment(f"TWPA_SNR/{SAMPLE}", sweep_obj, live_plot=True)

In [ ]:
data = qc.load_by_id(15) #2d sweep
twpa_freqs = np.array(data.get_data('TWPA_frequency')).reshape(-1)
twpa_powers = np.array(data.get_data('TWPA_power')).reshape(-1)
signal = np.array(data.get_data('SNR')).reshape(-1)
x = np.unique(twpa_freqs)
y = np.unique(twpa_powers)
z = signal.reshape(len(y),len(x))
X,Y = np.meshgrid(x,y)
plt.figure(figsize=(3, 3))
plt.pcolor(X,Y,z,cmap='viridis')
# plt.xlabel('TWPA Pump Power (dBm)')
# plt.ylabel('VNA Frequency (GHz)')
# #plt.title('ID = 309, TWPA Power = %.i'%(power))
cbar = plt.colorbar()
cbar.set_label('Magnitude (dB)')
# #plt.savefig("VNA_TWPA_309_%.i.png"%(power))
# #plt.axis([x.min(),x.max(),y.min(), y.max()])

In [ ]:
y0, x0 = np.unravel_index(np.argmax(z, axis=None), z.shape)
print(y[y0], x[x0], z[y0,x0])

In [ ]:
np.argmax(SNR)

In [ ]:
powers = np.arange(15,9,-1)
#for idnr, power in zip(ids,powers): 
data = qc.dataset.data_export.load_by_id(idnr)
freqs = data.get_data('frequency')
powers = data.get_data('twpa_pump_frequency')
signal = data.get_data('magnitude')


x = freqs[0][0]
y = np.array(powers)[:,0].T
X,Y = np.meshgrid(x,y)
Z = np.array(signal)[:,0,:]

#     # Znorm = Z
#     # for i in range(len(X[:,0])):
#     #     Znorm[i,:] = Z[i,:]/np.mean(Z[i,-15:-1])

#     plt.figure(figsize=(3, 3))
#     plt.pcolor(Y,X,Z,cmap='viridis', vmin=-60, vmax = 30)
#     plt.xlabel('TWPA Pump Frequency (GHz)')
#     plt.ylabel('VNA Frequency (GHz)')
#     plt.title('ID = %.i, TWPA Power = %.i'%(idnr,power))
#     cbar = plt.colorbar()
#     cbar.set_label('Magnitude (dB)')
#     plt.savefig("VNA_TWPA_%.i_%.i.png"%(idnr,power))
#     #plt.axis([x.min(),x.max(),y.min(), y.max()])

